# ****SOLUCIÓN:****

Se importan las librerias y se leen los datos escogidos: [Heart Disease UCI](https://www.kaggle.com/ronitf/heart-disease-uci)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor as mlp_r
from sklearn.neural_network import MLPClassifier as mlp_c
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import datasets

In [ ]:
Heart_data = pd.read_csv("../input/heart.csv")
Heart_data.head()

# REGRESIÓN

Se crea una matriz X que contiene todas las filas de los datos y un vector Y que contiene la información del tipo de dolor de pecho (cp)

In [ ]:
X = (Heart_data.iloc[:,:-1]).as_matrix()
y = (Heart_data.iloc[:,2]).as_matrix()

Se normalizan todos los datos en X segun el maximo y se hace lo mismo para Y (se divide entre 3 porque es el maximo de esos datos)

In [ ]:
X = (X - X.min(axis=0))/(X.max(axis=0) - X.min(axis=0))
y = y.reshape((-1,1))/3

Se dividen los datos para probar el modelo posteriormente

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

Entrenamos

In [ ]:
regressor = mlp_r(
    hidden_layer_sizes=(100,100),  activation='tanh', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
y_hats = regressor.predict(X_test)

In [ ]:
plt.scatter(y_test, y_hats, c='k')

plt.plot([0.1, 0.9], [0.1, 0.9], 'r')
plt.xlabel('Real')
plt.ylabel('Estimada')

Creamos 2 vectores con posibles numeros de neuronas en cada capa (2 capas) y probamos todas las combinaciones para posteriormente medir el error cuadratico medio de cada uno

In [ ]:
from sklearn.metrics import mean_squared_error

capa_1 = [5, 7, 9, 11, 13, 17,  19, 23, 29, 31]
capa_2 = [1, 5, 7, 9, 11, 13, 17,  19, 23, 29]

mse_m = np.zeros((len(capa_1),len(capa_1)))
mse_std = np.zeros((len(capa_1),len(capa_1)))

for j, n_1 in enumerate(capa_1):
    for k, n_2 in enumerate(capa_2):
        mse_temp = []
    
        for i in range(10):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
            regressor = mlp_r(hidden_layer_sizes=(n_1,n_2),  activation='tanh', solver='adam', alpha=0.001, batch_size='auto',
            learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
            random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
            early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            regressor.fit(X_train, y_train)
            y_hats = regressor.predict(X_test)
            mse_temp.append(mean_squared_error(y_test, y_hats))
            
        mse_m[j, k] = np.mean(mse_temp)
        mse_std[j, k] = np.std(mse_temp)


A continuación se grafican 2 mapas en donde se muestra el error cuadratico medio y la desviación estandar de cada red neuronal calculada anteriormente

In [ ]:
plt.imshow(mse_m)
plt.colorbar()

Para la grafica anterior, se puede ver que para el numero de neuronas en la capa 2 igual a 1 los datos contienen mucho error cuadratico medio, y que para 5 o mas neuronas en cada capa el error cuadratico medio es bajo y lo mismo ocurre para la desviación estandar. En general se puede decir que el modelo corre bien para 5 o mas neuronas en la capa 1 siempre y cuando tenga 5 o mas neuronas en la capa 2


In [ ]:
plt.imshow(mse_std)
plt.colorbar()

Variando el momentum con uno de los mejores resultados

In [ ]:
mmntm=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1]
mse_m = np.zeros(len(mmntm))
for m,n_3 in enumerate(mmntm):
    for i in range(21):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        regressor = mlp_r(hidden_layer_sizes=(29,31),  activation='tanh', solver='adam', alpha=0.001, batch_size='auto',
        learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
        random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=n_3, nesterovs_momentum=True,
        early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        regressor.fit(X_train, y_train)
        y_hats = regressor.predict(X_test)
        mse_temp.append(mean_squared_error(y_test, y_hats))
    mse_m[m] = np.mean(mse_temp)

In [ ]:
plt.plot(mmntm, mse_m, '.')
plt.xlabel('Momentum')
plt.ylabel('Error cuadratico medio')

Se puede ver que a medida que sube el momentum, el error cuadratico medio disminuye

# CLASIFICACIÓN
Se usan los mismos datos de [Heart Disease UCI](https://www.kaggle.com/ronitf/heart-disease-uci) y se dividen los datos para el entrenamiento justo como la regresión

In [ ]:
X = (Heart_data.iloc[:,:-1]).as_matrix()
y = (Heart_data.iloc[:,2]).as_matrix()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

Entrenamos

In [ ]:
clf = mlp_c(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(1000, 500), random_state=1)

clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)

Se calcula la matriz de confusion sin normalizar

In [ ]:
cm1 = confusion_matrix(y_test, y_hat)
cm2 = confusion_matrix(y_test, y_hat)
cm2 = cm2.astype('float') / cm2.sum(axis=1)[:, np.newaxis]
cm1

Para analizar la matriz, debemos fijarnos en la diagonal principal, para tener unos datos claros porcentualmente hablando, normalizamos.

In [ ]:
cm2

A continuación, se crean varios modelos con diferentes numeros de neuronas en cada capa, y por medio de la herramienta de densidad de imagen, se establece el mejor.

In [ ]:
capa_1=[100, 400, 800, 900, 1000]
capa_2=[20, 100, 500, 700, 800]
cero=np.zeros((len(capa_1),len(capa_2)))
uno=np.zeros((len(capa_1),len(capa_2)))
dos=np.zeros((len(capa_1),len(capa_2)))
tres=np.zeros((len(capa_1),len(capa_2)))
suma=np.zeros((len(capa_1),len(capa_2)))
for j, n_1 in enumerate(capa_1):
    for k, n_2 in enumerate(capa_2):
        X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42)

        clf = mlp_c(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(n_1, n_2), random_state=1)

        clf.fit(X_train, y_train)
        y_hat = clf.predict(X_test)
        cm2 = confusion_matrix(y_test, y_hat)
        cm2 = cm2.astype('float') / cm2.sum(axis=1)[:, np.newaxis]
        suma[j,k]=cm2[0,0]+cm2[1,1]+cm2[2,2]+cm2[3,3]
        cero[j,k]=cm2[0,0]
        uno[j,k]=cm2[1,1]
        dos[j,k]=cm2[2,2]
        tres[j,k]=cm2[3,3]
            


En cada imagen, se muestra la entrada diagonal de cada matriz de confusión normalizada, entre mas se aproxime a 1 mejor. Al final se tiene una grafica que cada entrada corresponde a un promedio de la diagonal.

In [ ]:
plt.imshow(cero)
plt.colorbar()

In [ ]:
plt.imshow(uno)
plt.colorbar()

In [ ]:
plt.imshow(dos)
plt.colorbar()

In [ ]:
plt.imshow(tres)
plt.colorbar()

In [ ]:
plt.imshow(suma/4)
plt.colorbar()

A medida que se aumenta el numero de neuronas de la capa 2, se obtienen mejores resultados, aunque el dato del cp categoria 3 nunca se puede predecir correctamente para este caso (en el unico caso que funciona, los demás fallan).